In [ ]:
import torch
from transformers import AutoModelForCausalLM
import os
from huggingface_hub import list_repo_files, snapshot_download

target_model_id = "mistralai/Mistral-7B-v0.1"

def download_transformers_model(repo_id, cache_dir=None):
    # Check for .safetensors files in the repository
    repo_files = list_repo_files(repo_id)
    has_safetensors = any(file.endswith('.safetensors') for file in repo_files)

    # Define ignore_patterns based on the presence of .safetensors files
    ignore_patterns = ["*.bin"] if has_safetensors else None

    # Download the repository, ignoring PyTorch .bin files if .safetensors files are present
    local_path = snapshot_download(repo_id=repo_id,
                                    cache_dir=cache_dir,
                                    ignore_patterns=ignore_patterns,
                                    )

    print(f"Model downloaded to: {local_path}")
    if has_safetensors:
        print("Note: PyTorch .bin files were ignored due to the presence of .safetensors files.")
    return os.path.abspath(local_path), has_safetensors


In [ ]:
import copy

target_model = AutoModelForCausalLM.from_pretrained(models["target"]["download_path"])

compressed_layer_num = 16

# Copy target model weights
base_model = copy.deepcopy(target_model)

In [ ]:
from peft.tuners.tuners_utils import replicate_layers

original_layers = [0, 15]
additional_layers = [15, 16] * compressed_layer_num

layer_map = [original_layers]
layer_map.extend(additional_layers)

replicate_layers(
    model = base_model,
    layer_map = layer_map
)

In [ ]:
base_model

In [ ]:
base_model.save_pretrained("./models/base/")

In [ ]:
# This code has been modified from its original version on the Axolotl project.
# Copyright 2023 Axolotl contributors.
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
# http://www.apache.org/licenses/LICENSE-2.0
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.


import os
import torch
import torch
import bitsandbytes as bnb
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
from peft.tuners.lora import QuantLinear


def get_linear_embedding_layers(model_type):
    """
    returns the linear embedding layers needed for loras, dependent on the model arch
    """
    if model_type == "gpt_neox":
        return ["embed_in", "embed_out"]
    if model_type == "falcon":
        return ["word_embeddings", "lm_head"]
    return ["embed_tokens", "lm_head"]


def find_all_linear_names(model):
    cls = (bnb.nn.Linear4bit, bnb.nn.Linear8bitLt, torch.nn.Linear, QuantLinear)

    names = []
    for name, module in model.named_modules():
        if (
            isinstance(module, cls)
            or "Linear" in module.__class__.__name__
            and module.__class__.__name__ not in ("LlamaLinearScalingRotaryEmbedding",)
        ):
            names.append(name)


    return names

def get_linear_module_names(model_id):
    model = AutoModelForCausalLM.from_pretrained(model_id, state_dict={}, device_map="meta") #avoid loading weights as we won't need them
    return find_all_linear_names(model)

linear_module_names = get_linear_module_names(models['target']['download_path'])

In [ ]:
base_linear_module_names = get_linear_module_names(models['base']['download_path'])

In [ ]:
base_linear_module_names

In [ ]:
import glob
from safetensors import safe_open

def load_safetensors(directory, framework="pt", device='cpu'):
    """
    Loads tensors from .safetensors files in the specified directory into a dictionary.

    Args:
    - directory (str): Path to the directory containing .safetensors files.
    - framework (str): The framework to use ('pt' for PyTorch, 'tf' for TensorFlow, etc.). Default is 'pt'.
    - device (str): The device to load the tensors on ('cpu', 'cuda', etc.). Default is 'cpu'.

    Returns:
    - dict: A dictionary containing all tensors from the .safetensors files.
    """
    tensors_dict = {}
    # Use glob to find all .safetensors files in the directory
    file_paths = glob.glob(f"{directory}/*.safetensors")

    # Loop through each file and load its tensors into the dictionary
    for file_path in sorted(file_paths):
        with safe_open(file_path, framework=framework, device=device) as f:
            for k in f.keys():
                tensors_dict[k] = f.get_tensor(k)

    return tensors_dict

In [ ]:
base_model_weights = load_safetensors(models['base']['download_path']) 
target_model_weights = load_safetensors(models['target']['download_path'])

In [ ]:
import torch

def _low_rank_decomposition(weight, reduced_rank=16):
    """
    Decompose a 2D matrix into low-rank matrices A and B using SVD.a

    :param weight: The matrix to decompose, of shape (H, W)
    :param reduced_rank: The final rank of the decomposition
    :return: A tuple of tensors (A, B)
    """
    if weight.dim() != 2:
        raise ValueError(f"Only support 2D matrix, but your input has {weight.dim()} dimensions.")

    # SVD Decomposition
    U, S, Vh = torch.linalg.svd(weight, full_matrices=False)

    # Truncated matrices
    A = Vh[:reduced_rank, :]
    B = U[:, :reduced_rank] @ torch.diag(S[:reduced_rank])

    return A, B

def decompose_delta_weight(new_weight, base_weight, alpha, reduced_rank, device=None):
    if device is None:
        device = 'cuda' if torch.cuda.is_available() else 'cpu'

    new_weight = new_weight.to(device)
    base_weight = base_weight.to(device)

    """
    Decompose the delta weight into low-rank matrices A and B, considering the alpha scaling factor.

    :param new_weight: The updated weight matrix after applying LoRA.
    :param base_weight: The original weight matrix before LoRA.
    :param alpha: The alpha scaling factor used in LoRA.
    :param reduced_rank: The rank for the low-rank decomposition.
    :return: A tuple of tensors (A, B)
    """
    delta_weight = new_weight - base_weight

    # Check if alpha is applied uniformly
    # Adjust the implementation if alpha is applied differently
    adjusted_delta_weight = delta_weight / alpha

    A, B = _low_rank_decomposition(adjusted_delta_weight, reduced_rank=reduced_rank)

    return A, B

In [ ]:
print(list(target_model_weights.keys()))
base_model

In [ ]:
from tqdm.notebook import tqdm

loras = {

}

# lower rank captures less of the original model, a rank of 32 is probably reasonable for small delta (task specific finetunes and such)
alpha = 1
rank = 32

for module in tqdm(linear_module_names):
  print(module)
  if module in target_model_weights.keys():
    target_tensor = target_model_weights[module+".weight"]
    base_tensor = base_model_weights[module+".weight"]

    lora_A, lora_B = decompose_delta_weight(target_tensor, base_tensor, alpha, rank)
    loras[f"base_model.model.{module}.lora_A.weight"] = lora_A.to('cpu')
    loras[f"base_model.model.{module}.lora_B.weight"] = lora_B.to('cpu')

In [ ]:
loras

In [ ]:
def get_module_peft_name(module_name):
    return module_name.split('.')[-1]

In [ ]:
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
from peft import get_peft_model, LoraConfig

LORA_OUT_DIR = "./lora"

lora_config = LoraConfig(
        lora_alpha=32, # Setting the alpha to the to decomposition rank value (instead of alpha value used) seems to give better performance. Further testing would be needed to understand what is the optimal alpha value to use
        lora_dropout=0,
        r=32,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules= list(set([get_module_peft_name(e) for e in linear_module_names])),
)

model = AutoModelForCausalLM.from_pretrained(models['base']['download_path'], load_in_4bit=True)

peft_model = get_peft_model(model, lora_config)

# Save to disk
peft_model.save_pretrained(LORA_OUT_DIR)

del peft_model

In [ ]:
import torch
from safetensors.torch import save_file


for key in loras.keys():
    loras[key] = loras[key].to('cpu').contiguous()

save_file(loras, os.path.join(LORA_OUT_DIR, 'adapter_model.safetensors'))

In [ ]:
import os
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM

config = PeftConfig.from_pretrained(os.path.abspath(LORA_OUT_DIR))
model = AutoModelForCausalLM.from_pretrained(models['base']['download_path'], load_in_4bit=True)
model = PeftModel.from_pretrained(model, os.path.abspath(LORA_OUT_DIR))

tokenizer = AutoTokenizer.from_pretrained(models['base']['download_path'], use_fast=True)

# Test input
messages = [
    {
        "role": "system",
        "content": "You are a human just going about your day.",
    },
    {"role": "user", "content": "Hey what's up?"},
]

# Tokenize and format the chat for the model
tokenized_chat = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt")

# Generate a response
outputs = model.generate(tokenized_chat, max_new_tokens=128)  # Adjust max_new_tokens if needed
response_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(response_text)